In [2]:
#diabetes df
import pandas as pd
import numpy as np
import pandas_profiling
import matplotlib.pyplot as plt
get_ipython().run_line_magic('matplotlib', 'inline')
import warnings # current version of seaborn generates a bunch of warnings that we'll ignore
warnings.filterwarnings("ignore")
import seaborn as sns
import sklearn
from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn import metrics as mt
from sklearn import preprocessing
from sklearn.model_selection import ShuffleSplit
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline

In [3]:
df_mlt = pd.read_csv('/Users/mtran/Documents/SMUDataSciences/2019-Summer/MiniLab/Diabetic_Cleaned.csv')
print("Finished loading data")

Finished loading data


In [4]:
#make dataframes for each column
ordinal = ["age", "time_in_hospital"]
categorical = ["race", "gender", "admission_type_id", "discharge_disposition_id", "admission_source_id", "medical_specialty", "max_glu_serum", "A1Cresult", "change", "diabetesMed", "diag_1_val", "diag_2_val", "diag_3_val", "meds_increased"]
continuous = ["num_lab_procedures", "num_procedures", "num_medications", "number_outpatient", "number_emergency", "number_inpatient", "number_diagnoses", "medication_count"]
medicines = ["metformin", "repaglinide", "nateglinide", "chlorpropamide", "glimepiride", "acetohexamide", "glipizide", "glyburide", "tolbutamide", "pioglitazone", "rosiglitazone", "acarbose", "miglitol", "troglitazone", "tolazamide", "insulin", "glyburide-metformin", "glipizide-metformin", "metformin-rosiglitazone", "metformin-pioglitazone"]
response = ["readmitted_tf"]

In [5]:
df_mlt_ordinal = df_mlt[ordinal]
df_mlt_categorical = df_mlt[categorical]
df_mlt_continuous = df_mlt[continuous]
df_mlt_medicines = df_mlt[medicines]
df_mlt_response = df_mlt[response]

In [6]:
#Make age ordinal
for i in range(0, 10):
    val = "[" + str(i*10) + "-" + str((i+1)*10) + ")"
    df_mlt_ordinal.loc[df_mlt_ordinal['age'] == val, 'age'] = i

In [7]:
#Make all categorical columns categorical
df_mlt_categorical['admission_type_id'] = df_mlt_categorical['admission_type_id'].astype(object)
df_mlt_categorical['discharge_disposition_id'] = df_mlt_categorical['discharge_disposition_id'].astype(object)
df_mlt_categorical['admission_source_id'] = df_mlt_categorical['admission_source_id'].astype(object)
df_mlt_categorical['admission_type_id'] = df_mlt_categorical['admission_type_id'].astype(object)
df_mlt_categorical['meds_increased'] = df_mlt_categorical['meds_increased'].astype(object)
print("Done")

Done


In [8]:
#Creating X and Y dataframes
df_mlt_x = pd.concat([df_mlt_ordinal, df_mlt_categorical, df_mlt_continuous], axis = 1)
df_mlt_med_x = pd.concat([df_mlt_ordinal, df_mlt_categorical, df_mlt_continuous, df_mlt_medicines], axis = 1)
df_mlt_hot_x = pd.concat([df_mlt_ordinal, pd.get_dummies(df_mlt_categorical), df_mlt_continuous], axis = 1)
df_mlt_hot_med_x = pd.concat([df_mlt_ordinal, pd.get_dummies(df_mlt_categorical), df_mlt_continuous, pd.get_dummies(df_mlt_medicines)], axis = 1)
df_mlt_y = df_mlt_response
print("Done")

Done


In [10]:
#Setting up training and test sets where x DOES NOT include medications
x_train, x_test, y_train, y_test = train_test_split(df_mlt_hot_x, df_mlt_y, test_size = 0.2, random_state=7)
#x_train.reset_index()
#y_train.reset_index()
#x_test.reset_index()
#y_test.reset_index()
x_train.shape, x_test.shape

((81412, 184), (20354, 184))

In [11]:
%%time
#Model 1: Logistic Regression where x DOES NOT include medications
#Sets up the object with different learning parameters and constants
model1 = LogisticRegression(penalty='l2', C=1.0, class_weight=None)
model1.fit(x_train, y_train)

C:\Users\mtran\AppData\Local\Continuum\anaconda3\envs\DS7331_Python3.5R\lib\site-packages\sklearn\utils\validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Wall time: 12.5 s


In [12]:
#Model 1: Prediction where x DOES NOT include medications
y_pred1 = model1.predict(x_test)

#Accuracy metrics
#Explains F1 score as function of Precision and Recall
#https://towardsdatascience.com/accuracy-precision-recall-or-f1-331fb37c5cb9
f1 = f1_score(y_test, y_pred1, average = 'weighted')
#Explains accuracy value
#https://skymind.ai/wiki/accuracy-precision-recall-f1
acc = accuracy_score(y_test, y_pred1)
#Explains how to read ROC value
#http://gim.unmc.edu/dxtests/roc3.htm
roc = roc_auc_score(y_test, model1.predict_proba(x_test)[:,1])
print("Model 1 - F1 Score:",f1)
print("Model 1 - Accuracy:",acc)
print("Model 1 - ROC:",roc)

Model 1 - F1 Score: 0.6665252283540175
Model 1 - Accuracy: 0.6761815859290558
Model 1 - ROC: 0.7414110274351416


In [13]:
#Model 2: Setting up training and test sets where x DOES include medications
x_train, x_test, y_train, y_test = train_test_split(df_mlt_hot_med_x, df_mlt_y, test_size = 0.2, random_state=7)
x_train.shape, x_test.shape

((81412, 251), (20354, 251))

In [14]:
#Model 2: Logistic Regression where x DOES include medications
#Sets up the object with different learning parameters and constants
model2 = LogisticRegression(penalty='l2', C=1.0, class_weight=None)
model2.fit(x_train, y_train)

C:\Users\mtran\AppData\Local\Continuum\anaconda3\envs\DS7331_Python3.5R\lib\site-packages\sklearn\utils\validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [15]:
#Model 2: Prediction where x DOES include medications
y_pred2 = model2.predict(x_test)

#Accuracy metrics
#Explains F1 score as function of Precision and Recall
#https://towardsdatascience.com/accuracy-precision-recall-or-f1-331fb37c5cb9
f1 = f1_score(y_test, y_pred2, average = 'weighted')
#Explains accuracy value
#https://skymind.ai/wiki/accuracy-precision-recall-f1
acc = accuracy_score(y_test, y_pred2)
#Explains how to read ROC value
#http://gim.unmc.edu/dxtests/roc3.htm
roc = roc_auc_score(y_test, model2.predict_proba(x_test)[:,1])
print("Model 2 - F1 Score:",f1)
print("Model 2 - Accuracy:",acc)
print("Model 2 - ROC:",roc)

Model 2 - F1 Score: 0.6660412565430712
Model 2 - Accuracy: 0.6752972388719662
Model 2 - ROC: 0.7419343116089305


In [16]:
#Model 3 - Normalize features
#Scale attributes by training set
scl_obj = StandardScaler()
scl_obj.fit(x_train)

x_train_scaled = scl_obj.transform(x_train) # apply to training
x_test_scaled = scl_obj.transform(x_test)

#Train the model just as before
lr_clf = LogisticRegression(penalty='l2', C=0.05) 
lr_clf.fit(x_train_scaled,y_train)

#Get test set predictions
y_pred3 = lr_clf.predict(x_test_scaled)

acc = mt.accuracy_score(y_test,y_pred3)
conf = mt.confusion_matrix(y_test,y_pred3)
print('Model 3 - Accuracy:', acc, '\nConfusion Matrix\n', conf)

#Sort these attributes and spit them out
zip_vars = zip(lr_clf.coef_.T,df_mlt_hot_x.columns) # combine attributes
zip_vars = sorted(zip_vars)
for coef, name in zip_vars:
    print(name, 'has weight of', coef[0]) # now print them out


C:\Users\mtran\AppData\Local\Continuum\anaconda3\envs\DS7331_Python3.5R\lib\site-packages\sklearn\preprocessing\data.py:617: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\mtran\AppData\Local\Continuum\anaconda3\envs\DS7331_Python3.5R\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
  
C:\Users\mtran\AppData\Local\Continuum\anaconda3\envs\DS7331_Python3.5R\lib\site-packages\ipykernel_launcher.py:7: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
  import sys
C:\Users\mtran\AppData\Local\Continuum\anaconda3\envs\DS7331_Python3.5R\lib\site-packages\sklearn\utils\validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel(

Model 3 - Accuracy: 0.6752972388719662 
Confusion Matrix
 [[8993 1976]
 [4633 4752]]
discharge_disposition_id_11 has weight of -0.8702271095203993
discharge_disposition_id_14 has weight of -0.17892145514780444
discharge_disposition_id_13 has weight of -0.1254620264976738
admission_source_id_2 has weight of -0.11633068178312078
admission_type_id_4 has weight of -0.1084907569720228
admission_type_id_3 has weight of -0.10231465135334027
admission_source_id_1 has weight of -0.08626716276123939
medical_specialty_InternalMedicine has weight of -0.07639696917733496
medical_specialty_Surgery-Cardiovascular/Thoracic has weight of -0.07540366772246755
admission_source_id_4 has weight of -0.06671542504108534
admission_source_id_6 has weight of -0.06291962967517713
admission_type_id_1 has weight of -0.06189753237494815
medical_specialty_Orthopedics-Reconstructive has weight of -0.05816551798797815
admission_source_id_17 has weight of -0.05472110369488204
medical_specialty_ObstetricsandGynecology h

In [17]:
#Model 4: Logistic Regression with SGD
#https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html#sklearn.linear_model.SGDClassifier
#https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
regularize_const = 0.1
iterations = 3

log_sgd = SGDClassifier(alpha = regularize_const,
        fit_intercept = True, l1_ratio = 0.0, learning_rate = 'optimal',
        loss = 'log', n_iter = iterations, n_jobs = -1, penalty = 'l2')

scl = StandardScaler()
log_sgd.fit(scl.fit_transform(x_train), y_train)
y_pred4 = log_sgd.predict(scl.transform(x_test))
conf = mt.confusion_matrix(y_test, y_pred4)
acc = mt.accuracy_score(y_test, y_pred4)
f1 = f1_score(y_test, y_pred4, average = 'weighted')

print("Model 4 - F1 Score:", f1)
print("Model 4 - Accuracy:", acc, '\nConfusion Matrix\n', conf)

C:\Users\mtran\AppData\Local\Continuum\anaconda3\envs\DS7331_Python3.5R\lib\site-packages\sklearn\preprocessing\data.py:617: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\mtran\AppData\Local\Continuum\anaconda3\envs\DS7331_Python3.5R\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\mtran\AppData\Local\Continuum\anaconda3\envs\DS7331_Python3.5R\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:130: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)
C:\Users\mtran\AppData\Local\Continuum\anaconda3\envs\DS7331_Python3.5R\lib\site-packages\sklearn\utils\validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array 

Model 4 - F1 Score: 0.6544941824792816
Model 4 - Accuracy: 0.6670924633978579 
Confusion Matrix
 [[9151 1818]
 [4958 4427]]


C:\Users\mtran\AppData\Local\Continuum\anaconda3\envs\DS7331_Python3.5R\lib\site-packages\ipykernel_launcher.py:13: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
  del sys.path[0]
